In [1]:
#importing libraries

import pandas as pd
import datetime
from datetime import date, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets
import warnings
warnings.filterwarnings("ignore")

#creating connections

scope= ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials= ServiceAccountCredentials.from_json_keyfile_name('car-master-sheet.json', scope) 
client= gspread.authorize(credentials)

clients = pygsheets.authorize(service_file='car-master-sheet.json')


#sheet keys

car_status_report='1CfqvArNmTofvNOFAhQ965B7cMA7lX40e3RZZKY6IUjI'
fleet_driver='1qceRS8LU17n5YWvgewcpJZif02KxV8Mtvs6anomHD80'
car_master='1_r5OMN1P8Tof5IRaE5jYd-jHaP8j15avQVMycDJdhec'
uber_ws='C:\\Users\\sagar\\Dropbox\\Dropbox\\DM Dashboard\\Master View.xlsx' 
dps_ws='C:\\Users\\sagar\\Dropbox\\Dropbox\\DM Dashboard\\Driver Performance Sheet.xlsx'
car_servicing_schedule_calling_servicing_tab='11WVBiisNIF8Xb7sx7GZEok7mG4Lz3uo96-e00xVtW0M'
commitment_mapping_3='1sy3Gxrnh8bX6ibpGR6X8rba8H2IfeSNdBGT7CIFi3SY'
wtd_sheet='1E9U_nG61vCYxbc3yQJHEKpIPfYuQxolllthf8weEckU'

#5 sheets keys

terrific='1f-DJ5O3zKKkAtXfXQghWzbsoXyy-ipdJ5q7Yma4tF9M'
roaring='1MiZZY9MPEhhg-B9LyfmThQGV3Dy3XnKpb5NqgaYFwIQ'
silent='1ZwXyZPyt7qhjTO5VkfwoGTJ48fmg_38MTdy4EHdVhwA'
deep='1TPRKPsQfy4qY19byAuYi0Ie3Xld7ddrrBS8xqhggtcM'
black='1KD0ABDWp3YvqIkmoQAZKRS-7-S8i-HtlO4KKpBMB1tU'
car_no_master_list='11D8_6u4ywy3yNYyrnMonyti6eflqUtpMfsxgLMBXEzk'

##################################################sheets extraction#######################################################

#car_status_report
sheet= client.open_by_key(car_status_report)
ws= sheet.worksheet('Mumbai')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dfs=dfm.loc[dfm['Type'].isin(['Revenue Share'])]
dfs['Current DM']=dfs['Current DM'].str.title()
reject_df=dfs.loc[dfs['Current DM'] != 'Pune']

sheet= client.open_by_key(car_no_master_list)
ws= sheet.worksheet('DM')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
mumbai_60_40=dfm[dfm['Cities']=='Mumbai 60:40']

all_team=reject_df[reject_df['Current DM'].isin(mumbai_60_40['DM NAME'])]
car_status_df=all_team.iloc[:,[1,0,4]]
car_status_df['ETM']=car_status_df['ETM'].str.strip()
car_status_df.rename({'Current DM':'Team Name','Car Number':'car_number'},axis=1,inplace=True)
car_status_df

#fleet_column_df
fleet_column_df=all_team.iloc[:,[0,5]]
fleet_column_df.rename({'Car Number':'car_number','Fleet ID':'Fleet'},axis=1,inplace=True)
fleet_column_df.fillna('',inplace=True)
fleet_column_df

#fleet_lead_col_df

fleet_lead_col_df=all_team.iloc[:,[0,6]]
fleet_lead_col_df.rename({'Car Number':'car_number','Fleet Lead':'Fleet_Lead'},axis=1,inplace=True)
fleet_lead_col_df.fillna('',inplace=True)
fleet_lead_col_df

#uber_df_ND_column

uber_df= pd.read_excel(uber_ws,sheet_name=-1)#uber sheet

uber_df_nd_df=uber_df.iloc[:,[0,3,4,5,6,7,8,9]]
today = datetime.date.today()
today_timestmp_column = today.strftime("%Y-%m-%d %H:%M:%S")
yesterday = today- timedelta(days=1)
yesterday_timestmp_column=yesterday.strftime("%Y-%m-%d %H:%M:%S")
monday = today - datetime.timedelta(days=today.weekday())
monday_timestmp_column=monday.strftime("%Y-%m-%d %H:%M:%S")
if today==monday:
    uber_column_nd=uber_df_nd_df.iloc[:,[0]]
    uber_column_nd['ND']='0'
    uber_column_nd.rename({'Car No':'car_number'},axis=1,inplace=True)
    uber_column_nd
else:
    column_range=pd.date_range(monday_timestmp_column,yesterday_timestmp_column)
    uber_column=pd.concat([uber_df['Car No'],uber_df[column_range.to_list()]],axis=1)
    df__ = []
    for i in uber_column.index:        
        df__.append(list(uber_column[column_range.to_list()].values[i]).count('ND'))
    uber_column['ND'] = df__
    uber_column_nd=uber_column.loc[:,['Car No','ND']]
    uber_column_nd.rename({'Car No':'car_number'},axis=1,inplace=True)
    uber_column_nd
    
#uber_df_today_yesterday

today = datetime.date.today()
monday = today - datetime.timedelta(days=today.weekday())
today_timestmp_column = today.strftime("%Y-%m-%d %H:%M:%S")
ttc=today_timestmp_column+'.1'
yesterday = today- timedelta(days=1)
yesterday_timestmp_column=yesterday.strftime("%Y-%m-%d %H:%M:%S")
ytc=yesterday_timestmp_column+'.1'
if today==monday:
    uber_df[ytc]='0'
    uber_dfs=uber_df.loc[:,['Car No','Last Week Trips',(ytc),'Total Trips','Balance','Hours Online','Net Fare']]
    uber_dfs.rename({'Car No':'car_number',(ytc):'Yesterdays Trips','Balance':'Balanced Trips','Hours Online':'Online Hours','Net Fare':'Revenue Total'},axis=1,inplace=True)
    uber_dfs
else:
    uber_dfs=uber_df.loc[:,['Car No','Last Week Trips',(ytc),'Total Trips','Balance','Hours Online','Net Fare']]
    uber_dfs.rename({'Car No':'car_number',(ytc):'Yesterdays Trips','Balance':'Balanced Trips','Hours Online':'Online Hours','Net Fare':'Revenue Total'},axis=1,inplace=True)
    uber_dfs

#car_servicing_schedule_calling_servicing_tab

sheet= client.open_by_key(car_servicing_schedule_calling_servicing_tab)
ws= sheet.worksheet('Calling Servicing Pending')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
servicing_df=dfm.iloc[:,[1,0]]
servicing_df.rename({'Car Number':'car_number','':'Servicing'},axis=1,inplace=True)
servicing_df

#dps_dead_km_yesterday

dps_dfs = pd.read_excel(dps_ws,sheet_name=0)#dps sheet

today = datetime.date.today()
today_timestmp_column=today.strftime("%d-%m-%Y")
yesterday = today- timedelta(days=1)
yesterday_timestmp_column=yesterday.strftime("%Y-%m-%d")
dps_dfs['Date']=pd.to_datetime(dps_dfs['Date'])
df_thisweek = dps_dfs[(dps_dfs['Date'] == yesterday_timestmp_column)]
dead_km_yesterday=df_thisweek.loc[:,['Car no','Dead KMs']]
dead_km_yesterday.rename({'Car no':'car_number','Dead KMs':'Dead KM yesterday'},axis=1,inplace=True)
dead_km_yesterday

#dps_dead_km_this_week

previous_monday = today - datetime.timedelta(days=today.weekday())
dps_dfs['Date']=pd.to_datetime(dps_dfs['Date'])
df_this_week = dps_dfs[(dps_dfs['Date'] >= pd.to_datetime(previous_monday)) & (dps_dfs['Date'] < pd.to_datetime(today))]
dps_dead_km_this_week=df_this_week.iloc[:,[1,8]]
dps_dead_km_this_week['Dead KMs']=pd.to_numeric(dps_dead_km_this_week['Dead KMs'], errors = 'coerce')
dfs=dps_dead_km_this_week.groupby(['Car no'],as_index=True)['Dead KMs'].sum()
dkmtw=pd.DataFrame(dfs).reset_index()
dkmtw.rename({'Car no':'car_number','Dead KMs':'This week Dead KM'},axis=1,inplace=True)
dkmtw

#dps_dead_km_previous_monday_to_previous_sunday

today = datetime.date.today()
previous_monday = today - datetime.timedelta(days=today.weekday())
previous_monday_week= previous_monday - datetime.timedelta(days=7)
previous_sunday_week= previous_monday - datetime.timedelta(days=1)
dps_dfs['Date']=pd.to_datetime(dps_dfs['Date'])
df_this_week = dps_dfs[(dps_dfs['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_dfs['Date'] <= pd.to_datetime(previous_sunday_week))]
dps_dead_km_this_week=df_this_week.iloc[:,[1,8]]
dps_dead_km_this_week['Dead KMs']=pd.to_numeric(dps_dead_km_this_week['Dead KMs'], errors = 'coerce')
dfs=dps_dead_km_this_week.groupby(['Car no'],as_index=True)['Dead KMs'].sum()
Last_Week_Dead_KM=pd.DataFrame(dfs).reset_index()
Last_Week_Dead_KM.rename({'Car no':'car_number','Dead KMs':'Last Week Dead KM'},axis=1,inplace=True)
Last_Week_Dead_KM

#uber_df_accecptance_cancelation

uber_df_a_c_df=uber_df.loc[:,['Car No','Acceptance','Cancellation']]
uber_df_a_c_df['Acceptance']=uber_df_a_c_df['Acceptance']*100
uber_df_a_c_df['Cancellation']=uber_df_a_c_df['Cancellation']*100
uber_df_a_c_df['Acceptance'] = uber_df_a_c_df['Acceptance'].astype(str).replace('\.\d+', '%', regex=True)
uber_df_a_c_df['Cancellation']=uber_df_a_c_df['Cancellation'].astype(str).replace('\.\d+', '%', regex=True)
uber_df_a_c_df.rename({'Car No':'car_number'},axis=1,inplace=True)
uber_df_a_c_df['Acceptance']=uber_df_a_c_df['Acceptance'].str.replace('nan','')
uber_df_a_c_df['Cancellation']=uber_df_a_c_df['Cancellation'].str.replace('nan','')
uber_df_a_c_df

# uber_df_a_c_df=uber_df.loc[:,['Car No','Acceptance','Cancellation']]
# uber_df_a_c_df.rename({'Car No':'car_number'},axis=1,inplace=True)
# uber_df_a_c_df.fillna('',inplace=True)
# uber_df_a_c_df

# ############################################################Car status pushing############################################################

# #car_status_df 2 columns

# car_status_df_2_column=car_status_df.iloc[:,[0,1]]
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='B',end='C')
# ws.set_dataframe(car_status_df_2_column,(1,2))

# #car_status_df 1 columns

# car_status_df_1_column=car_status_df.iloc[:,[2]]
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='E',end='E')
# ws.set_dataframe(car_status_df_1_column,(1,5))

# # # ################################################# mapping car  status on the basis of car number and pushing ##################################################################

# # fleet_lead_col_df

# cs_cmdf=car_status_df.merge(fleet_lead_col_df, on='car_number',how='left')
# cs_cmdf_df=cs_cmdf.iloc[:,[3]]
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='A',end='A')
# ws.set_dataframe(cs_cmdf_df,(1,1))

# # fleet_column_df

# cs_atm3=car_status_df.merge(fleet_column_df, on='car_number',how='left')
# cs_atm3_df=cs_atm3.iloc[:,[3]]
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='D',end='D')
# ws.set_dataframe(cs_atm3_df,(1,4))

# # #uber nd column

# cs_uber_nd=car_status_df.merge(uber_column_nd, on='car_number',how='left')
# cs_uber_nd_df=cs_uber_nd.iloc[:,[3]]
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='M',end='M')
# ws.set_dataframe(cs_uber_nd_df,(1,13))

# #uber_dfs

# cs_uber_dfs=car_status_df.merge(uber_dfs, on='car_number',how='left')
# cs_uber_dfs_df=cs_uber_dfs.iloc[:,[3,4,5,6,7,8]]
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='N',end='S')
# ws.set_dataframe(cs_uber_dfs_df,(1,14))

# #car_servicing_schedule_calling_servicing

# cs_servicing=car_status_df.merge(servicing_df, on='car_number',how='left')
# cs_servicing_d=cs_servicing.iloc[:,[3]]
# cs_servicing_df=cs_servicing_d.fillna('')
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='U',end='U')
# ws.set_dataframe(cs_servicing_df,(1,21))

# #dead km yesterday

# dead_km_yesterday_df=car_status_df.merge(dead_km_yesterday, on='car_number',how='left')
# cs_dky_df=dead_km_yesterday_df.iloc[:,[3]]
# cs_dky=cs_dky_df.fillna('')
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='W',end='W')
# ws.set_dataframe(cs_dky,(1,23))

# #dkmtw

# dkmtw_df=car_status_df.merge(dkmtw, on='car_number',how='left')
# cs_dkmtw=dkmtw_df.iloc[:,[3]]
# cs_dkmtw_df=cs_dkmtw.fillna('')
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='X',end='X')
# ws.set_dataframe(cs_dkmtw_df,(1,24))

# #Last_Week_Dead_KM

# lwdk_df=car_status_df.merge(Last_Week_Dead_KM, on='car_number',how='left')
# lwdk_dfs=lwdk_df.iloc[:,[3]]
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='Y',end='Y')
# ws.set_dataframe(lwdk_dfs,(1,25))

# #uber_df_accecptance_cancelation

# uber_df_acceptnce_cancellation=car_status_df.merge(uber_df_a_c_df, on='car_number',how='left')
# uber_df_acceptnce_cancellation_df=uber_df_acceptnce_cancellation.iloc[:,[3,4]]
# uber_df_acceptnce_cancellation_dfs=uber_df_acceptnce_cancellation_df.fillna('')
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='AA',end='AB')
# ws.set_dataframe(uber_df_acceptnce_cancellation_dfs,(1,27))

# # # #######################################################pushing uber to uber all sheets

# #uber

# uber_dfs_tab=uber_df.iloc[:,[0,1,2,39,3,4,5,6,7,8,9,45,17,18,19,20,21,22,23,44,24,25,26,27,28,29,30]]
# uber_dfs_tab.columns=uber_dfs_tab.columns.astype(str)
# uber_dfs_tab
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_dfs_tab,(1,1))

# #terrific
# uber_df_terrific= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Terrific Tigers"])]
# uber_df_terrific.columns=uber_df_terrific.columns.astype(str)
# uber_df_terrific
# sheet= clients.open_by_key(terrific)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_terrific,(1,1))

# #roaring

# uber_df_roaring= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Roaring Lions"])]
# uber_df_roaring.columns=uber_df_roaring.columns.astype(str)
# uber_df_roaring
# sheet= clients.open_by_key(roaring)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_roaring,(1,1))

# # #silent

# uber_df_silent= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Silent Killers"])]
# uber_df_silent.columns=uber_df_silent.columns.astype(str)
# uber_df_silent
# sheet= clients.open_by_key(silent)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_silent,(1,1))

# # #deep

# uber_df_deep= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Deep Hunters"])]
# uber_df_deep.columns=uber_df_deep.columns.astype(str)
# uber_df_deep
# sheet= clients.open_by_key(deep)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_deep,(1,1))

# # #black

# uber_df_black= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Black Panthers"])]
# uber_df_black.columns=uber_df_black.columns.astype(str)
# uber_df_black
# sheet= clients.open_by_key(black)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_black,(1,1))


# # ######################################################################################dps to alls sheets

# #dps

# today = datetime.date.today()
# previous_monday = today - datetime.timedelta(days=today.weekday())
# previous_monday_week= today - datetime.timedelta(days=14)
# dps_dfs['Date']=pd.to_datetime(dps_dfs['Date'])
# df_this_week = dps_dfs[(dps_dfs['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_dfs['Date'] <= pd.to_datetime(today))]
# sheet= clients.open_by_key(commitment_mapping_3)#master
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week,(1,1))

# #terrific

# dps_df_terrific= dps_dfs[dps_dfs["DM name"].isin(["Terrific Tigers"])]
# dps_df_terrific['Date']=pd.to_datetime(dps_df_terrific['Date'])
# df_this_week_terrefic = dps_df_terrific[(dps_df_terrific['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_terrific['Date'] <= pd.to_datetime(today))]
# df_this_week_terrefic
# sheet= clients.open_by_key(terrific)
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_terrefic,(1,1))

# # #roaring

# dps_df_roaring= dps_dfs[dps_dfs["DM name"].isin(["Roaring Lions"])]
# dps_df_roaring['Date']=pd.to_datetime(dps_df_roaring['Date'])
# df_this_week_roaring = dps_df_roaring[(dps_df_roaring['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_roaring['Date'] <= pd.to_datetime(today))]
# df_this_week_roaring
# sheet= clients.open_by_key(roaring)
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_roaring,(1,1))

# #silent 

# dps_df_silent= dps_dfs[dps_dfs["DM name"].isin(["Silent Killers"])]
# dps_df_silent['Date']=pd.to_datetime(dps_df_silent['Date'])
# df_this_week_silent = dps_df_silent[(dps_df_silent['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_silent['Date'] <= pd.to_datetime(today))]
# df_this_week_silent
# sheet= clients.open_by_key(silent)
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_silent,(1,1))

# #deep

# dps_df_deep= dps_dfs[dps_dfs["DM name"].isin(["Deep Hunters"])]
# dps_df_deep['Date']=pd.to_datetime(dps_df_deep['Date'])
# df_this_week_deep = dps_df_deep[(dps_df_deep['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_deep['Date'] <= pd.to_datetime(today))]
# df_this_week_deep
# sheet= clients.open_by_key(deep)
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_deep,(1,1))

# # #black

# dps_df_black= dps_dfs[dps_dfs["DM name"].isin(["Black Panthers"])]
# dps_df_black['Date']=pd.to_datetime(dps_df_black['Date'])
# df_this_week_black = dps_df_black[(dps_df_black['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_black['Date'] <= pd.to_datetime(today))]
# df_this_week_black
# sheet= clients.open_by_key(black )
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_black,(1,1))

# # ########################################################################uber_dps_dead km to dead km all sheets

# #uber_deadkm

# uber_df.columns=uber_df.columns.astype(str)
# uber_dps_df=uber_df.iloc[:,[0,1,2,50,31,32,33,34,35,36,37]]
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dps_df,(1,1))

# # #terrific 

# uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Terrific Tigers"])]
# sheet= clients.open_by_key(terrific)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

# # #roaring

# uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Roaring Lions"])]
# sheet= clients.open_by_key(roaring)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

# # #silent

# uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Silent Killers"])]
# sheet= clients.open_by_key(silent)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

# # #deep 

# uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Deep Hunters"])]
# sheet= clients.open_by_key(deep)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

# # #black

# uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Black Panthers"])]
# sheet= clients.open_by_key(black)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

# # ########################################################pushing to 5 sheets###########################################

# #terrific

# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)

# terific_dfs = dfm[dfm["Team Name"].isin(["Terrific Tigers"])]
# terific_df=terific_dfs.iloc[:,0:19]
# sheet= clients.open_by_key(terrific)
# ws= sheet.worksheet_by_title('Terrific_Tigers')
# ws.clear(start='A',end='S')
# ws.set_dataframe(terific_df,(1,1))


# terific_dfs = dfm[dfm["Team Name"].isin(["Terrific Tigers"])]
# terific_df_1=terific_dfs.iloc[:,20:28]
# sheet= clients.open_by_key(terrific)
# ws= sheet.worksheet_by_title('Terrific_Tigers')
# ws.clear(start='U',end='AB')
# ws.set_dataframe(terific_df_1,(1,21))

# #roaring

# Roaring_dfs= dfm[dfm["Team Name"].isin(["Roaring Lions"])]
# Roaring_df=Roaring_dfs.iloc[:,0:19]
# sheet= clients.open_by_key(roaring)
# ws= sheet.worksheet_by_title('Roaring_Lions')
# ws.clear(start='A',end='S')
# ws.set_dataframe(Roaring_df,(1,1))

# Roaring_dfs= dfm[dfm["Team Name"].isin(["Roaring Lions"])]
# Roaring_df_1=Roaring_dfs.iloc[:,20:28]
# sheet= clients.open_by_key(roaring)
# ws= sheet.worksheet_by_title('Roaring_Lions')
# ws.clear(start='U',end='AB')
# ws.set_dataframe(Roaring_df_1,(1,21))

# #silent

# Silent_dfs= dfm[dfm["Team Name"].isin(["Silent Killers"])]
# Silent_df=Silent_dfs.iloc[:,0:19]
# sheet= clients.open_by_key(silent)
# ws= sheet.worksheet_by_title('Silent_Killers')
# ws.clear(start='A',end='S')
# ws.set_dataframe(Silent_df,(1,1))

# Silent_dfs= dfm[dfm["Team Name"].isin(["Silent Killers"])]
# Silent_df_1=Silent_dfs.iloc[:,20:28]
# sheet= clients.open_by_key(silent)
# ws= sheet.worksheet_by_title('Silent_Killers')
# ws.clear(start='U',end='AB')
# ws.set_dataframe(Silent_df_1,(1,21))

# # deep

# Deep_Hunters_dfs= dfm[dfm["Team Name"].isin(["Deep Hunters"])]
# Deep_Hunters_df=Deep_Hunters_dfs.iloc[:,0:19]
# sheet= clients.open_by_key(deep)
# ws= sheet.worksheet_by_title('Deep_Hunters')
# ws.clear(start='A',end='S')
# ws.set_dataframe(Deep_Hunters_df,(1,1))

# Deep_Hunters_df= dfm[dfm["Team Name"].isin(["Deep Hunters"])]
# Deep_Hunters_df_1=Deep_Hunters_df.iloc[:,20:28]
# sheet= clients.open_by_key(deep)
# ws= sheet.worksheet_by_title('Deep_Hunters')
# ws.clear(start='U',end='AB')
# ws.set_dataframe(Deep_Hunters_df_1,(1,21))

# # black

# Black_Panthers_dfs= dfm[dfm["Team Name"].isin(["Black Panthers"])]
# Black_Panthers_df=Black_Panthers_dfs.iloc[:,0:19]
# sheet= clients.open_by_key(black)
# ws= sheet.worksheet_by_title('Black_Panthers')
# ws.clear(start='A',end='S')
# ws.set_dataframe(Black_Panthers_df,(1,1))

# Black_Panthers_df= dfm[dfm["Team Name"].isin(["Black Panthers"])]
# Black_Panthers_df_1=Black_Panthers_df.iloc[:,20:28]
# sheet= clients.open_by_key(black)
# ws= sheet.worksheet_by_title('Black_Panthers')
# ws.clear(start='U',end='AB')
# ws.set_dataframe(Black_Panthers_df_1,(1,21))

# #wtd trips script

# wtd_df=uber_df[['Total Trips','Car No']]
# sheet= clients.open_by_key(wtd_sheet)
# ws= sheet.worksheet_by_title('Trips_Data')
# ws.set_dataframe(wtd_df,(1,1))
# print("wtd trips data tab updated succesfully")

# print("Car mapping car no updated succesfully")

,car_number,Acceptance,Cancellation
0,MH02FG6166,0%,
1,MH02FG6195,36%,6%
2,MH02FG6196,85%,5%
3,MH02FG6197,86%,0%
4,MH02FG6199,100%,2%
...,...,...,...
1547,MH01DR9808,100%,0%
1548,MH01DR9810,80%,4%
1549,MH01DR9811,84%,2%
1550,MH01DR9812,87%,0%


In [2]:
uber_df_a_c_df['Cancellation'].unique()

array(['', '6%', '5%', '0%', '2%', '9%', '3%', '4%', '8%', '12%', '17%',
       '7%', '33%', '13%', '10%', '1%', '11%', '19%', '24%', '14%', '15%',
       '22%', '25%', '16%', '21%', '20%', '18%', '28%', '36%', '60%',
       '40%', '30%', '37%'], dtype=object)

# Rough work

In [12]:
sheet= client.open_by_key(car_servicing_schedule_calling_servicing_tab)
ws= sheet.worksheet('Calling Servicing Pending')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
servicing_df=dfm.iloc[:,[1,0]]
servicing_df.rename({'Car Number':'car_number','':'Servicing'},axis=1,inplace=True)
servicing_df

,car_number,Servicing
0,MH01CV5218,Service Due
1,MH48F3514,Service Due
2,MH01DR4120,Service Due
3,MH01DR4555,Service Due
4,MH01CV7785,Service Due
...,...,...
745,MH01CV9808,Service Due
746,MH01DR1846,Service Due
747,MH01DR2476,Service Due
748,MH01DR2543,Service Due


In [19]:
#changes sameer sir
uber_dfs_tab=uber_df.iloc[:,[0,1,2,39,3,4,5,6,7,8,9,45,17,18,19,20,21,22,23,44,24,25,26,27,28,29,30]]
uber_dfs_tab

,Car No,DM,Fleet,Total Trips,2022-08-01 00:00:00,2022-08-02 00:00:00,2022-08-03 00:00:00,2022-08-04 00:00:00,2022-08-05 00:00:00,2022-08-06 00:00:00,...,2022-08-06 00:00:00.2,2022-08-07 00:00:00.2,Net Fare,2022-08-01 00:00:00.3,2022-08-02 00:00:00.3,2022-08-03 00:00:00.3,2022-08-04 00:00:00.3,2022-08-05 00:00:00.3,2022-08-06 00:00:00.3,2022-08-07 00:00:00.3
0,MH02FG6166,Deep Hunters,everest18@11,0,ND,ND,ND,ND,ND,ND,...,0,0,0.00,0.00,0,0,0,0,0,0
1,MH02FG6195,Black Panthers,everest969@,0,ND,ND,ND,ND,ND,ND,...,0,0,0.00,0.00,0,0,0,0,0,0
2,MH02FG6196,Black Panthers,everest969@,9,9,ND,ND,ND,ND,ND,...,0,0,1487.50,1487.50,0,0,0,0,0,0
3,MH02FG6197,Black Panthers,everest969@,9,9,ND,ND,ND,ND,ND,...,0,0,1891.44,1891.44,0,0,0,0,0,0
4,MH02FG6199,Black Panthers,everest969@,10,10,ND,ND,ND,ND,ND,...,0,0,2318.38,2318.38,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,MH01DR9541,Roaring Lions,everest25@175,7,7,ND,ND,ND,ND,ND,...,0,0,1746.96,1746.96,0,0,0,0,0,0
1535,MH01DR9539,Red Eagle,0,0,ND,ND,ND,ND,ND,ND,...,0,0,0.00,0.00,0,0,0,0,0,0
1536,MH01DR9536,Silent Killers,everest17@90,0,ND,ND,ND,ND,ND,ND,...,0,0,0.00,0.00,0,0,0,0,0,0
1537,MH01DR9537,Silent Killers,everest17@90,0,ND,ND,ND,ND,ND,ND,...,0,0,0.00,0.00,0,0,0,0,0,0


In [3]:
uber_dfs

,car_number,Last Week Trips,Yesterdays Trips,Total Trips,Balanced Trips,Online Hours,Revenue Total
0,MH02FG6166,31,0,0,50,0.00,0.00
1,MH02FG6195,81,11,28,22,37.77,5632.64
2,MH02FG6196,17,3,13,37,22.85,2348.68
3,MH02FG6197,85,4,28,22,29.79,5206.41
4,MH02FG6199,64,7,28,22,32.62,5918.59
...,...,...,...,...,...,...,...
1547,MH01DR9808,52,0,10,40,12.18,2322.09
1548,MH01DR9810,21,4,16,34,41.86,4704.41
1549,MH01DR9811,47,12,23,27,22.53,4596.73
1550,MH01DR9812,66,5,20,30,30.45,4509.22


In [4]:
uber_df

,Car No,DM,Fleet,2022-08-22 00:00:00,2022-08-23 00:00:00,2022-08-24 00:00:00,2022-08-25 00:00:00,2022-08-26 00:00:00,2022-08-27 00:00:00,2022-08-28 00:00:00,...,Cancellation,Dead KM Deviation,2022-08-22 00:00:00.5,2022-08-23 00:00:00.5,2022-08-24 00:00:00.5,2022-08-25 00:00:00.5,2022-08-26 00:00:00.5,2022-08-27 00:00:00.5,2022-08-28 00:00:00.5,Model
0,MH02FG6166,Deep Hunters,everest18@11,ND,ND,ND,ND,ND,ND,ND,...,NaN,46.10,0.00,0.00,0.00,0,0,0,0,celerio
1,MH02FG6195,Black Panthers,everest969@,11,6,11,ND,ND,ND,ND,...,0.06,-480.19,129.30,69.99,151.90,0,0,0,0,celerio
2,MH02FG6196,Black Panthers,everest969@,6,4,3,ND,ND,ND,ND,...,0.05,121.16,67.58,24.67,46.24,0,0,0,0,celerio
3,MH02FG6197,Black Panthers,everest969@,11,13,4,ND,ND,ND,ND,...,0.00,-31.60,112.51,145.93,37.59,0,0,0,0,celerio
4,MH02FG6199,Black Panthers,everest969@,16,5,7,ND,ND,ND,ND,...,0.02,-17.62,132.99,83.19,156.98,0,0,0,0,celerio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,MH01DR9808,Silent Killers,everest17@90,ND,10,ND,ND,ND,ND,ND,...,0.00,-196.11,0.00,151.11,0.00,0,0,0,0,0
1548,MH01DR9810,Silent Killers,everest17@90,7,5,4,ND,ND,ND,ND,...,0.04,2.10,128.61,118.29,103.20,0,0,0,0,0
1549,MH01DR9811,Silent Killers,everest17@90,11,ND,12,ND,ND,ND,ND,...,0.02,-348.48,135.48,0.00,114.00,0,0,0,0,0
1550,MH01DR9812,Silent Killers,everest17@90,8,7,5,ND,ND,ND,ND,...,0.00,-385.58,101.65,83.00,95.93,0,0,0,0,0


# wtd trips scripts

In [30]:
import pandas as pd
import pygsheets
import warnings
warnings.filterwarnings("ignore")


wtd_sheet='1E9U_nG61vCYxbc3yQJHEKpIPfYuQxolllthf8weEckU'

wtd_df=uber_df[['Total Trips','Car No']]
wtd_df
sheet= clients.open_by_key(wtd_sheet)
ws= sheet.worksheet_by_title('Trips_Data')
ws.set_dataframe(wtd_df,(1,1))

,Total Trips,Car No
0,14,MH02FG6166
1,23,MH02FG6195
2,19,MH02FG6196
3,31,MH02FG6197
4,19,MH02FG6199
...,...,...
1534,25,MH01DR9541
1535,0,MH01DR9539
1536,0,MH01DR9536
1537,0,MH01DR9537


In [5]:
#car_status_df 2 columns

car_status_df_2_column=car_status_df.iloc[:,[0,1]]
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='B',end='C')
ws.set_dataframe(car_status_df_2_column,(1,2))

#car_status_df 1 columns

car_status_df_1_column=car_status_df.iloc[:,[2]]
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column,(1,5))

# # ################################################# mapping car  status on the basis of car number and pushing ##################################################################

# fleet_lead_col_df

cs_cmdf=car_status_df.merge(fleet_lead_col_df, on='car_number',how='left')
cs_cmdf_df=cs_cmdf.iloc[:,[3]]
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='A',end='A')
ws.set_dataframe(cs_cmdf_df,(1,1))

# fleet_column_df

cs_atm3=car_status_df.merge(fleet_column_df, on='car_number',how='left')
cs_atm3_df=cs_atm3.iloc[:,[3]]
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='D',end='D')
ws.set_dataframe(cs_atm3_df,(1,4))

# #uber nd column

cs_uber_nd=car_status_df.merge(uber_column_nd, on='car_number',how='left')
cs_uber_nd_df=cs_uber_nd.iloc[:,[3]]
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='M',end='M')
ws.set_dataframe(cs_uber_nd_df,(1,13))

#uber_dfs

cs_uber_dfs=car_status_df.merge(uber_dfs, on='car_number',how='left')
cs_uber_dfs_df=cs_uber_dfs.iloc[:,[3,4,5,6,7,8]]
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='N',end='S')
ws.set_dataframe(cs_uber_dfs_df,(1,14))

#car_servicing_schedule_calling_servicing

cs_servicing=car_status_df.merge(servicing_df, on='car_number',how='left')
cs_servicing_d=cs_servicing.iloc[:,[3]]
cs_servicing_df=cs_servicing_d.fillna('')
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='U',end='U')
ws.set_dataframe(cs_servicing_df,(1,21))

#dead km yesterday

dead_km_yesterday_df=car_status_df.merge(dead_km_yesterday, on='car_number',how='left')
cs_dky_df=dead_km_yesterday_df.iloc[:,[3]]
cs_dky=cs_dky_df.fillna('')
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='W',end='W')
ws.set_dataframe(cs_dky,(1,23))

#dkmtw

dkmtw_df=car_status_df.merge(dkmtw, on='car_number',how='left')
cs_dkmtw=dkmtw_df.iloc[:,[3]]
cs_dkmtw_df=cs_dkmtw.fillna('')
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='X',end='X')
ws.set_dataframe(cs_dkmtw_df,(1,24))

#Last_Week_Dead_KM

lwdk_df=car_status_df.merge(Last_Week_Dead_KM, on='car_number',how='left')
lwdk_dfs=lwdk_df.iloc[:,[3]]
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='Y',end='Y')
ws.set_dataframe(lwdk_dfs,(1,25))

#uber_df_accecptance_cancelation

uber_df_acceptnce_cancellation=car_status_df.merge(uber_df_a_c_df, on='car_number',how='left')
uber_df_acceptnce_cancellation_df=uber_df_acceptnce_cancellation.iloc[:,[3,4]]
uber_df_acceptnce_cancellation_dfs=uber_df_acceptnce_cancellation_df.fillna('')
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='AA',end='AB')
ws.set_dataframe(uber_df_acceptnce_cancellation_dfs,(1,27))

# # #######################################################pushing uber to uber all sheets

#uber

uber_dfs_tab=uber_df.iloc[:,[0,1,2,39,3,4,5,6,7,8,9,45,17,18,19,20,21,22,23,44,24,25,26,27,28,29,30]]
uber_dfs_tab.columns=uber_dfs_tab.columns.astype(str)
uber_dfs_tab
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Uber')
ws.clear()
ws.set_dataframe(uber_dfs_tab,(1,1))

#terrific
uber_df_terrific= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Terrific Tigers"])]
uber_df_terrific.columns=uber_df_terrific.columns.astype(str)
uber_df_terrific
sheet= clients.open_by_key(terrific)
ws= sheet.worksheet_by_title('Uber')
ws.clear()
ws.set_dataframe(uber_df_terrific,(1,1))

#roaring

uber_df_roaring= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Roaring Lions"])]
uber_df_roaring.columns=uber_df_roaring.columns.astype(str)
uber_df_roaring
sheet= clients.open_by_key(roaring)
ws= sheet.worksheet_by_title('Uber')
ws.clear()
ws.set_dataframe(uber_df_roaring,(1,1))

# #silent

uber_df_silent= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Silent Killers"])]
uber_df_silent.columns=uber_df_silent.columns.astype(str)
uber_df_silent
sheet= clients.open_by_key(silent)
ws= sheet.worksheet_by_title('Uber')
ws.clear()
ws.set_dataframe(uber_df_silent,(1,1))

# #deep

uber_df_deep= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Deep Hunters"])]
uber_df_deep.columns=uber_df_deep.columns.astype(str)
uber_df_deep
sheet= clients.open_by_key(deep)
ws= sheet.worksheet_by_title('Uber')
ws.clear()
ws.set_dataframe(uber_df_deep,(1,1))

# #black

uber_df_black= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Black Panthers"])]
uber_df_black.columns=uber_df_black.columns.astype(str)
uber_df_black
sheet= clients.open_by_key(black)
ws= sheet.worksheet_by_title('Uber')
ws.clear()
ws.set_dataframe(uber_df_black,(1,1))


# ######################################################################################dps to alls sheets

#dps

today = datetime.date.today()
previous_monday = today - datetime.timedelta(days=today.weekday())
previous_monday_week= today - datetime.timedelta(days=14)
dps_dfs['Date']=pd.to_datetime(dps_dfs['Date'])
df_this_week = dps_dfs[(dps_dfs['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_dfs['Date'] <= pd.to_datetime(today))]
sheet= clients.open_by_key(commitment_mapping_3)#master
ws= sheet.worksheet_by_title('DPS')
ws.clear()
ws.set_dataframe(df_this_week,(1,1))

#terrific

dps_df_terrific= dps_dfs[dps_dfs["DM name"].isin(["Terrific Tigers"])]
dps_df_terrific['Date']=pd.to_datetime(dps_df_terrific['Date'])
df_this_week_terrefic = dps_df_terrific[(dps_df_terrific['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_terrific['Date'] <= pd.to_datetime(today))]
df_this_week_terrefic
sheet= clients.open_by_key(terrific)
ws= sheet.worksheet_by_title('DPS')
ws.clear()
ws.set_dataframe(df_this_week_terrefic,(1,1))

# #roaring

dps_df_roaring= dps_dfs[dps_dfs["DM name"].isin(["Roaring Lions"])]
dps_df_roaring['Date']=pd.to_datetime(dps_df_roaring['Date'])
df_this_week_roaring = dps_df_roaring[(dps_df_roaring['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_roaring['Date'] <= pd.to_datetime(today))]
df_this_week_roaring
sheet= clients.open_by_key(roaring)
ws= sheet.worksheet_by_title('DPS')
ws.clear()
ws.set_dataframe(df_this_week_roaring,(1,1))

#silent 

dps_df_silent= dps_dfs[dps_dfs["DM name"].isin(["Silent Killers"])]
dps_df_silent['Date']=pd.to_datetime(dps_df_silent['Date'])
df_this_week_silent = dps_df_silent[(dps_df_silent['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_silent['Date'] <= pd.to_datetime(today))]
df_this_week_silent
sheet= clients.open_by_key(silent)
ws= sheet.worksheet_by_title('DPS')
ws.clear()
ws.set_dataframe(df_this_week_silent,(1,1))

#deep

dps_df_deep= dps_dfs[dps_dfs["DM name"].isin(["Deep Hunters"])]
dps_df_deep['Date']=pd.to_datetime(dps_df_deep['Date'])
df_this_week_deep = dps_df_deep[(dps_df_deep['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_deep['Date'] <= pd.to_datetime(today))]
df_this_week_deep
sheet= clients.open_by_key(deep)
ws= sheet.worksheet_by_title('DPS')
ws.clear()
ws.set_dataframe(df_this_week_deep,(1,1))

# #black

dps_df_black= dps_dfs[dps_dfs["DM name"].isin(["Black Panthers"])]
dps_df_black['Date']=pd.to_datetime(dps_df_black['Date'])
df_this_week_black = dps_df_black[(dps_df_black['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_black['Date'] <= pd.to_datetime(today))]
df_this_week_black
sheet= clients.open_by_key(black )
ws= sheet.worksheet_by_title('DPS')
ws.clear()
ws.set_dataframe(df_this_week_black,(1,1))

# ########################################################################uber_dps_dead km to dead km all sheets

#uber_deadkm

uber_df.columns=uber_df.columns.astype(str)
uber_dps_df=uber_df.iloc[:,[0,1,2,50,31,32,33,34,35,36,37]]
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Dead KM')
ws.clear()
ws.set_dataframe(uber_dps_df,(1,1))

# #terrific 

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Terrific Tigers"])]
sheet= clients.open_by_key(terrific)
ws= sheet.worksheet_by_title('Dead KM')
ws.clear()
ws.set_dataframe(uber_dfs_dead,(1,1))

# #roaring

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Roaring Lions"])]
sheet= clients.open_by_key(roaring)
ws= sheet.worksheet_by_title('Dead KM')
ws.clear()
ws.set_dataframe(uber_dfs_dead,(1,1))

# #silent

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Silent Killers"])]
sheet= clients.open_by_key(silent)
ws= sheet.worksheet_by_title('Dead KM')
ws.clear()
ws.set_dataframe(uber_dfs_dead,(1,1))

# #deep 

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Deep Hunters"])]
sheet= clients.open_by_key(deep)
ws= sheet.worksheet_by_title('Dead KM')
ws.clear()
ws.set_dataframe(uber_dfs_dead,(1,1))

# #black

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Black Panthers"])]
sheet= clients.open_by_key(black)
ws= sheet.worksheet_by_title('Dead KM')
ws.clear()
ws.set_dataframe(uber_dfs_dead,(1,1))

# ########################################################pushing to 5 sheets###########################################

#terrific

sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)

terific_dfs = dfm[dfm["Team Name"].isin(["Terrific Tigers"])]
terific_df=terific_dfs.iloc[:,0:19]
sheet= clients.open_by_key(terrific)
ws= sheet.worksheet_by_title('Terrific_Tigers')
ws.clear(start='A',end='S')
ws.set_dataframe(terific_df,(1,1))


terific_dfs = dfm[dfm["Team Name"].isin(["Terrific Tigers"])]
terific_df_1=terific_dfs.iloc[:,20:28]
sheet= clients.open_by_key(terrific)
ws= sheet.worksheet_by_title('Terrific_Tigers')
ws.clear(start='U',end='AB')
ws.set_dataframe(terific_df_1,(1,21))

#roaring

Roaring_dfs= dfm[dfm["Team Name"].isin(["Roaring Lions"])]
Roaring_df=Roaring_dfs.iloc[:,0:19]
sheet= clients.open_by_key(roaring)
ws= sheet.worksheet_by_title('Roaring_Lions')
ws.clear(start='A',end='S')
ws.set_dataframe(Roaring_df,(1,1))

Roaring_dfs= dfm[dfm["Team Name"].isin(["Roaring Lions"])]
Roaring_df_1=Roaring_dfs.iloc[:,20:28]
sheet= clients.open_by_key(roaring)
ws= sheet.worksheet_by_title('Roaring_Lions')
ws.clear(start='U',end='AB')
ws.set_dataframe(Roaring_df_1,(1,21))

#silent

Silent_dfs= dfm[dfm["Team Name"].isin(["Silent Killers"])]
Silent_df=Silent_dfs.iloc[:,0:19]
sheet= clients.open_by_key(silent)
ws= sheet.worksheet_by_title('Silent_Killers')
ws.clear(start='A',end='S')
ws.set_dataframe(Silent_df,(1,1))

Silent_dfs= dfm[dfm["Team Name"].isin(["Silent Killers"])]
Silent_df_1=Silent_dfs.iloc[:,20:28]
sheet= clients.open_by_key(silent)
ws= sheet.worksheet_by_title('Silent_Killers')
ws.clear(start='U',end='AB')
ws.set_dataframe(Silent_df_1,(1,21))

# deep

Deep_Hunters_dfs= dfm[dfm["Team Name"].isin(["Deep Hunters"])]
Deep_Hunters_df=Deep_Hunters_dfs.iloc[:,0:19]
sheet= clients.open_by_key(deep)
ws= sheet.worksheet_by_title('Deep_Hunters')
ws.clear(start='A',end='S')
ws.set_dataframe(Deep_Hunters_df,(1,1))

Deep_Hunters_df= dfm[dfm["Team Name"].isin(["Deep Hunters"])]
Deep_Hunters_df_1=Deep_Hunters_df.iloc[:,20:28]
sheet= clients.open_by_key(deep)
ws= sheet.worksheet_by_title('Deep_Hunters')
ws.clear(start='U',end='AB')
ws.set_dataframe(Deep_Hunters_df_1,(1,21))

# black

Black_Panthers_dfs= dfm[dfm["Team Name"].isin(["Black Panthers"])]
Black_Panthers_df=Black_Panthers_dfs.iloc[:,0:19]
sheet= clients.open_by_key(black)
ws= sheet.worksheet_by_title('Black_Panthers')
ws.clear(start='A',end='S')
ws.set_dataframe(Black_Panthers_df,(1,1))

Black_Panthers_df= dfm[dfm["Team Name"].isin(["Black Panthers"])]
Black_Panthers_df_1=Black_Panthers_df.iloc[:,20:28]
sheet= clients.open_by_key(black)
ws= sheet.worksheet_by_title('Black_Panthers')
ws.clear(start='U',end='AB')
ws.set_dataframe(Black_Panthers_df_1,(1,21))

#wtd trips script
wtd_df=uber_df[['Total Trips','Car No']]
sheet= clients.open_by_key(wtd_sheet)
ws= sheet.worksheet_by_title('Trips_Data')
ws.set_dataframe(wtd_df,(1,1))

print("Car mapping car no updated succesfully")

Car mapping car no updated succesfully


In [21]:
#importing libraries

import pandas as pd
import datetime
from datetime import date, timedelta
import pygsheets
import warnings
warnings.filterwarnings("ignore")

#creating connections

clients = pygsheets.authorize(service_file='client_secret.json')

#sheet keys

commitment_mapping_3='1sy3Gxrnh8bX6ibpGR6X8rba8H2IfeSNdBGT7CIFi3SY'
servicing_calling_mumbai='1tdBXqs_kSEfl1sGk9gEEEUSgRcCTqAUDw7r13-7FmzA'

#5 sheets keys

terrific='1f-DJ5O3zKKkAtXfXQghWzbsoXyy-ipdJ5q7Yma4tF9M'
roaring='1MiZZY9MPEhhg-B9LyfmThQGV3Dy3XnKpb5NqgaYFwIQ'
silent='1ZwXyZPyt7qhjTO5VkfwoGTJ48fmg_38MTdy4EHdVhwA'
deep='1TPRKPsQfy4qY19byAuYi0Ie3Xld7ddrrBS8xqhggtcM'
black='1KD0ABDWp3YvqIkmoQAZKRS-7-S8i-HtlO4KKpBMB1tU'


sheet= clients.open_by_key(servicing_calling_mumbai)
ws= sheet.worksheet_by_title('Confirm Appointment List (DM)')
data = ws.get_all_values()
headers = data.pop(2)
master_servicing_df= pd.DataFrame(data,columns=headers)
master_servicing=master_servicing_df[master_servicing_df['DM Name'].isin(['Deep Hunters', 'Silent Killers', 'Terrific Tigers', 'Black Panthers', 'Roaring Lions'])]
master_servicing

sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Servicing')
ws.clear(start='A',end='L')
ws.set_dataframe(master_servicing,(1,1))

terific_servicing = master_servicing[master_servicing["DM Name"].isin(["Terrific Tigers"])]
sheet= clients.open_by_key(terrific)
ws= sheet.worksheet_by_title('Servicing')
ws.clear(start='A',end='L')
ws.set_dataframe(terific_servicing,(1,1))

roaring_servicing = master_servicing[master_servicing["DM Name"].isin(["Roaring Lions"])]
sheet= clients.open_by_key(roaring)
ws= sheet.worksheet_by_title('Servicing')
ws.clear(start='A',end='L')
ws.set_dataframe(roaring_servicing,(1,1))

silent_servicing = master_servicing[master_servicing["DM Name"].isin(["Silent Killers"])]
sheet= clients.open_by_key(silent)
ws= sheet.worksheet_by_title('Servicing')
ws.clear(start='A',end='L')
ws.set_dataframe(silent_servicing,(1,1))

deep_servicing = master_servicing[master_servicing["DM Name"].isin(["Deep Hunters"])]
sheet= clients.open_by_key(deep)
ws= sheet.worksheet_by_title('Servicing')
ws.clear(start='A',end='L')
ws.set_dataframe(deep_servicing,(1,1))

black_servicing = master_servicing[master_servicing["DM Name"].isin(["Black Panthers"])]
sheet= clients.open_by_key(black)
ws= sheet.worksheet_by_title('Servicing')
ws.clear(start='A',end='L')
ws.set_dataframe(black_servicing,(1,1))

In [19]:
master_servicing['DM Name'].unique()

array(['Deep Hunters', 'Silent Killers', 'Roaring Lions',
       'Black Panthers', 'Terrific Tigers'], dtype=object)

In [8]:
car_status_df['Team Name'].unique()

array(['Deep Hunters', 'Silent Killers', 'Terrific Tigers',
       'Black Panthers', 'Roaring Lions'], dtype=object)

In [4]:
#fleet_column_df
fleet_column_df=all_team.iloc[:,[0,5]]
fleet_column_df.rename({'Car Number':'car_number','Fleet ID':'Fleet'},axis=1,inplace=True)
fleet_column_df.fillna('',inplace=True)
fleet_column_df

# #fleet_lead_col_df

# fleet_lead_col_df=all_team.iloc[:,[0,6]]
# fleet_lead_col_df.rename({'Car Number':'car_number','Fleet Lead':'Fleet_Lead'},axis=1,inplace=True)
# fleet_lead_col_df.fillna('',inplace=True)
# fleet_lead_col_df

,car_number,Fleet
70,MH01CV4378,everest18@11
78,MH01CV4503,everest17@90
84,MH01CV4827,everest510@
86,MH01CV4829,everest969@
108,MH01CV5162,everest25@175
...,...,...
2556,MH02FG6235,everest969@
2557,MH02FG6236,everest510@
2558,MH02FG6248,everest15@15
2559,MH02FG6249,everest15@15


In [ ]:
#uber_df_ND_column

uber_df= pd.read_excel(uber_ws,sheet_name=-1)#uber sheet

uber_df_nd_df=uber_df.iloc[:,[0,3,4,5,6,7,8,9]]
today = datetime.date.today()
today_timestmp_column = today.strftime("%Y-%m-%d %H:%M:%S")
yesterday = today- timedelta(days=1)
yesterday_timestmp_column=yesterday.strftime("%Y-%m-%d %H:%M:%S")
monday = today - datetime.timedelta(days=today.weekday())
monday_timestmp_column=monday.strftime("%Y-%m-%d %H:%M:%S")
if today==monday:
    uber_column_nd=uber_df_nd_df.iloc[:,[0]]
    uber_column_nd['ND']='0'
    uber_column_nd.rename({'Car No':'car_number'},axis=1,inplace=True)
    uber_column_nd
else:
    column_range=pd.date_range(monday_timestmp_column,yesterday_timestmp_column)
    uber_column=pd.concat([uber_df['Car No'],uber_df[column_range.to_list()]],axis=1)
    df__ = []
    for i in uber_column.index:        
        df__.append(list(uber_column[column_range.to_list()].values[i]).count('ND'))
    uber_column['ND'] = df__
    uber_column_nd=uber_column.loc[:,['Car No','ND']]
    uber_column_nd.rename({'Car No':'car_number'},axis=1,inplace=True)
    uber_column_nd

In [2]:
sheet= client.open_by_key(car_status_report)
ws= sheet.worksheet('Cars')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dfs=dfm.loc[dfm['Type'].isin(['Revenue Share'])]
dfs['Current DM']=dfs['Current DM'].str.title()
reject_df=dfs.loc[dfs['Current DM'] != 'Pune']
# reject_df_1=reject_df.loc[reject_df['Current DM'] != 'Ev']
all_team=reject_df[reject_df['Current DM'].isin(['Deep Hunters', 'Silent Killers', 'Terrific Tigers', 'Black Panthers', 'Roaring Lions'])]
car_status_df=all_team.iloc[:,[1,0,4]]
car_status_df['ETM']=car_status_df['ETM'].str.strip()
car_status_df.rename({'Current DM':'Team Name','Car Number':'car_number'},axis=1,inplace=True)
print(car_status_df)

            Team Name  car_number       ETM
70       Deep Hunters  MH01CV4378  ETM23266
78     Silent Killers  MH01CV4503  ETM05062
84    Terrific Tigers  MH01CV4827   Parking
86     Black Panthers  MH01CV4829   Parking
108     Roaring Lions  MH01CV5162   Parking
...               ...         ...       ...
2556   Black Panthers  MH02FG6235  ETM20574
2557  Terrific Tigers  MH02FG6236  ETM18321
2558    Roaring Lions  MH02FG6248  ETM10990
2559    Roaring Lions  MH02FG6249  ETM18575
2560  Terrific Tigers  MH02FG6481  ETM14068

[1426 rows x 3 columns]


In [ ]:
#uber

uber_dfs_tab=uber_df.iloc[:,[0,1,2,39,10,11,12,13,14,15,16,45,17,18,19,20,21,22,23,44,24,25,26,27,28,29,30]]
uber_dfs_tab.columns=uber_dfs_tab.columns.astype(str)
uber_dfs_tab
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_dfs,(1,1))

#terrific
uber_df_terrific= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Terrific Tigers"])]
uber_df_terrific.columns=uber_df_terrific.columns.astype(str)
uber_df_terrific
# sheet= clients.open_by_key(terrific)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_terrific,(1,1))

#roaring

uber_df_roaring= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Roaring Lions"])]
uber_df_roaring.columns=uber_df_roaring.columns.astype(str)
uber_df_roaring
# sheet= clients.open_by_key(roaring)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_roaring,(1,1))

# #silent

uber_df_silent= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Silent Killers"])]
uber_df_silent.columns=uber_df_silent.columns.astype(str)
uber_df_silent
# sheet= clients.open_by_key(silent)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_silent,(1,1))

# #deep

uber_df_deep= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Deep Hunters"])]
uber_df_deep.columns=uber_df_deep.columns.astype(str)
uber_df_deep
# sheet= clients.open_by_key(deep)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_deep,(1,1))

# #black

uber_df_black= uber_dfs_tab[uber_dfs_tab["DM"].isin(["Black Panthers"])]
uber_df_black.columns=uber_df_black.columns.astype(str)
uber_df_black
# sheet= clients.open_by_key(black)
# ws= sheet.worksheet_by_title('Uber')
# ws.clear()
# ws.set_dataframe(uber_df_black,(1,1))


,Date,Car no,KMs,OnTime,DM name,Duty,UberKMs,Trips,Dead KMs,Allowance,Deviation,remark
0,2022-05-21,MH01CV5986,756.25,959.000000,Vinay Bhai leasing,ND,0.00,0,756.25,0.0,756.25,NaN
1,2022-05-21,MH01CV4885,612.87,924.000000,Deadly shark,ND,0.00,0,612.87,0.0,612.87,NaN
2,2022-05-21,MH01CV5724,509.85,1218.000000,Deadly shark,ND,0.00,0,509.85,0.0,509.85,NaN
3,2022-05-21,MH01CV4380,505.87,1367.000000,Deadly shark,ND,0.00,0,505.87,0.0,505.87,NaN
4,2022-05-21,MH01CV3947,491.86,1004.000000,Red Eagle,ND,0.00,0,491.86,0.0,491.86,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
208996,2022-07-15,TS09UD5423,391.20,897.449483,0,Uber,321.52,15,69.68,60.0,9.68,NaN
208997,2022-07-15,TS09UD4916,209.00,670.865367,0,Uber,140.44,13,68.56,54.0,14.56,NaN
208998,2022-07-15,TS09UD5407,211.40,592.785383,0,Uber,148.59,9,62.81,42.0,20.81,NaN
208999,2022-07-15,TS09UD4899,181.40,638.688117,0,Uber,139.62,9,41.78,42.0,-0.22,NaN


In [ ]:
#dps

today = datetime.date.today()
previous_monday = today - datetime.timedelta(days=today.weekday())
previous_monday_week= today - datetime.timedelta(days=14)
dps_dfs['Date']=pd.to_datetime(dps_dfs['Date'])
df_this_week = dps_dfs[(dps_dfs['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_dfs['Date'] <= pd.to_datetime(today))]
# sheet= clients.open_by_key(commitment_mapping_3)#master
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week,(1,1))

#terrific

dps_df_terrific= dps_dfs[dps_dfs["DM name"].isin(["Terrific Tigers"])]
dps_df_terrific['Date']=pd.to_datetime(dps_df_terrific['Date'])
df_this_week_terrefic = dps_df_terrific[(dps_df_terrific['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_terrific['Date'] <= pd.to_datetime(today))]
df_this_week_terrefic
# sheet= clients.open_by_key(terrific)
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_terrefic,(1,1))

# #roaring

dps_df_roaring= dps_dfs[dps_dfs["DM name"].isin(["Roaring Lions"])]
dps_df_roaring['Date']=pd.to_datetime(dps_df_roaring['Date'])
df_this_week_roaring = dps_df_roaring[(dps_df_roaring['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_roaring['Date'] <= pd.to_datetime(today))]
df_this_week_roaring
# sheet= clients.open_by_key(roaring)
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_roaring,(1,1))

#silent 

dps_df_silent= dps_dfs[dps_dfs["DM name"].isin(["Silent Killers"])]
dps_df_silent['Date']=pd.to_datetime(dps_df_silent['Date'])
df_this_week_silent = dps_df_silent[(dps_df_silent['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_silent['Date'] <= pd.to_datetime(today))]
df_this_week_silent
# sheet= clients.open_by_key(silent)
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_silent,(1,1))

#deep

dps_df_deep= dps_dfs[dps_dfs["DM name"].isin(["Deep Hunters"])]
dps_df_deep['Date']=pd.to_datetime(dps_df_deep['Date'])
df_this_week_deep = dps_df_deep[(dps_df_deep['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_deep['Date'] <= pd.to_datetime(today))]
df_this_week_deep
# sheet= clients.open_by_key(deep)
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_deep,(1,1))

# #black

dps_df_black= dps_dfs[dps_dfs["DM name"].isin(["Black Panthers"])]
dps_df_black['Date']=pd.to_datetime(dps_df_black['Date'])
df_this_week_black = dps_df_black[(dps_df_black['Date'] >= pd.to_datetime(previous_monday_week)) & (dps_df_black['Date'] <= pd.to_datetime(today))]
df_this_week_black
# sheet= clients.open_by_key(black )
# ws= sheet.worksheet_by_title('DPS')
# ws.clear()
# ws.set_dataframe(df_this_week_black,(1,1))

,Date,Car no,KMs,OnTime,DM name,Duty,UberKMs,Trips,Dead KMs,Allowance,Deviation,remark
152000,2022-07-02,MH02FG6197,271.26,763.000000,Black Panthers,Uber,203.42,8,67.84,39.0,28.84,NaN
152004,2022-07-02,MH02FG6199,261.26,840.000000,Black Panthers,Uber,179.59,5,81.67,30.0,51.67,NaN
152025,2022-07-02,MH02FG6196,210.46,811.000000,Black Panthers,Uber,169.14,20,41.32,75.0,-33.68,NaN
152028,2022-07-02,MH02FG6202,207.32,845.000000,Black Panthers,Uber,166.90,12,40.42,51.0,-10.58,NaN
152069,2022-07-02,MH02FG6235,160.34,661.000000,Black Panthers,Uber,129.49,8,30.85,39.0,-8.15,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
207339,2022-07-15,MH01DR9180,0.00,0.000000,Black Panthers,ND,0.00,0,0.00,0.0,0.00,NaN
207358,2022-07-15,MH01DR9130,24.40,0.000000,Black Panthers,ND,0.00,0,24.40,0.0,24.40,NaN
207360,2022-07-15,MH01DR9128,25.00,0.000000,Black Panthers,ND,0.00,0,25.00,0.0,25.00,NaN
208855,2022-07-15,MH01DR8455,170.60,691.454267,Black Panthers,Uber,84.12,8,86.48,39.0,47.48,NaN


In [ ]:
#car_status_report

sheet= client.open_by_key(car_status_report)
ws= sheet.worksheet('Cars')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dfs=dfm.loc[dfm['Type'].isin(['Revenue Share'])]
dfs['Current DM']=dfs['Current DM'].str.title()
reject_df=dfs.loc[dfs['Current DM'] != 'Pune']
reject_df_1=reject_df.loc[reject_df['Current DM'] != 'Ev'] 
all_team=reject_df_1[reject_df_1['Current DM'].isin(['Deep Hunters', 'Silent Killers', 'Terrific Tigers', 'Black Panthers', 'Roaring Lions'])]
all_team
# car_status_df=all_team.iloc[:,[1,0,4]]
# car_status_df['ETM']=car_status_df['ETM'].str.strip()
# car_status_df.rename({'Current DM':'Team Name','Car Number':'car_number'},axis=1,inplace=True)
# car_status_df

In [ ]:
#aatmanirbhar3
# all_car_details='C:\\Users\\sagar\\Dropbox\\DM Dashboard\\All Cars Details.xlsx'
# all_car_details_df=pd.read_excel(all_car_details,sheet_name='Fleet_Lead')
# aatmanirbhar_3_df=all_team.iloc[:,[0,5]]
# aatmanirbhar_3_df.rename({'Car Number':'car_number','Fleet ID':'Fleet'},axis=1,inplace=True)
# aatmanirbhar_3_df.fillna('',inplace=True)
# aatmanirbhar_3_df

#commitment mapping Dashboard

# all_car_details='C:\\Users\\sagar\\Dropbox\\DM Dashboard\\All Cars Details.xlsx'
# all_car_details_df=pd.read_excel(all_car_details,sheet_name='Fleet_Lead')
commitment_mapping_dashboard_df=all_team.iloc[:,[0,6]]
commitment_mapping_dashboard_df.rename({'Car Number':'car_number','Fleet Lead':'Fleet_Lead'},axis=1,inplace=True)
commitment_mapping_dashboard_df.fillna('',inplace=True)
commitment_mapping_dashboard_df

In [ ]:
uber_df

,Car No,DM,Fleet,2022-07-11 00:00:00,2022-07-12 00:00:00,2022-07-13 00:00:00,2022-07-14 00:00:00,2022-07-15 00:00:00,2022-07-16 00:00:00,2022-07-17 00:00:00,...,Cancellation,Dead KM Deviation,2022-07-11 00:00:00.5,2022-07-12 00:00:00.5,2022-07-13 00:00:00.5,2022-07-14 00:00:00.5,2022-07-15 00:00:00.5,2022-07-16 00:00:00.5,2022-07-17 00:00:00.5,Model
0,MH02FG6165,Deep Hunters,everest18@11,10,7,4,7,2,ND,ND,...,0.04,135.49,158.05,99.04,57.44,114.97,62.96,0,0,celerio
1,MH02FG6166,Deep Hunters,everest18@11,ND,ND,ND,6,ND,ND,ND,...,0.00,146.96,0.00,0.00,0.00,188.45,0.00,0,0,celerio
2,MH02FG6195,Black Panthers,everest969@,ND,14,13,12,13,ND,ND,...,0.03,-857.58,0.00,166.68,174.28,123.46,177.16,0,0,celerio
3,MH02FG6196,Black Panthers,everest969@,ND,5,3,8,7,ND,ND,...,0.07,-43.14,0.00,66.88,78.69,102.90,65.27,0,0,celerio
4,MH02FG6197,Black Panthers,everest969@,ND,10,13,8,11,ND,ND,...,0.04,128.12,0.00,176.38,184.05,140.89,155.96,0,0,celerio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,MH01DR9018,Terrific Tigers,everest455@,10,8,6,7,7,ND,ND,...,0.06,-12.14,209.60,87.80,81.81,146.46,133.17,0,0,spresso
1331,MH01DR9020,Terrific Tigers,everest20@01,11,12,3,ND,3,ND,ND,...,0.00,114.74,189.97,129.11,92.17,0.00,125.11,0,0,spresso
1332,MH01DR9021,Terrific Tigers,everest20@01,8,4,ND,ND,ND,ND,ND,...,0.11,-249.15,123.49,59.66,0.00,0.00,0.00,0,0,spresso
1333,MH01DR9028,Silent Killers,everest17@90,ND,ND,8,1,10,ND,ND,...,0.02,-19.03,0.00,0.00,119.59,35.09,193.05,0,0,spresso


In [ ]:
#uber_deadkm

uber_df.columns=uber_df.columns.astype(str)
uber_dps_df=uber_df.iloc[:,[0,1,2,50,31,32,33,34,35,36,37]]
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dps_df,(1,1))

# #terrific 

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Terrific Tigers"])]
# sheet= clients.open_by_key(terrific)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

# #roaring

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Roaring Lions"])]
# sheet= clients.open_by_key(roaring)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

# #silent

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Silent Killers"])]
# sheet= clients.open_by_key(silent)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

# #deep 

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Deep Hunters"])]
# sheet= clients.open_by_key(deep)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

# #black

uber_dfs_dead= uber_dps_df[uber_dps_df["DM"].isin(["Black Panthers"])]
# sheet= clients.open_by_key(black)
# ws= sheet.worksheet_by_title('Dead KM')
# ws.clear()
# ws.set_dataframe(uber_dfs_dead,(1,1))

,Car No,DM,Fleet,Dead KM Deviation,2022-07-11 00:00:00.4,2022-07-12 00:00:00.4,2022-07-13 00:00:00.4,2022-07-14 00:00:00.4,2022-07-15 00:00:00.4,2022-07-16 00:00:00.4,2022-07-17 00:00:00.4
11,MH02FG6248,Roaring Lions,Everest15@15,31.35,64.16,-25.21,-1.57,-21.49,15.46,0,0
12,MH02FG6249,Roaring Lions,Everest15@15,67.62,9.13,33.58,5.08,6.23,13.60,0,0
18,MH01CV5162,Roaring Lions,Everest15@15,0.00,0.00,0.00,0.00,0.00,0.00,0,0
19,MH01CV5164,Roaring Lions,Everest15@15,357.89,48.57,24.83,67.03,187.06,30.40,0,0
24,MH01CV6141,Roaring Lions,Everest15@15,35.44,0.00,27.20,24.90,-4.61,-12.05,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1244,MH01DR8842,Roaring Lions,everest15@15,102.24,29.37,16.39,14.09,13.50,28.89,0,0
1245,MH01DR8843,Roaring Lions,everest15@15,50.28,63.21,12.63,-23.90,8.91,-10.57,0,0
1246,MH01DR8844,Roaring Lions,everest15@15,-25.91,12.55,-18.27,-8.92,-0.96,-10.31,0,0
1247,MH01DR8845,Roaring Lions,everest15@15,111.61,2.92,18.00,104.10,-3.16,-10.25,0,0


In [ ]:
reject_df_1['Current DM'].unique()

In [ ]:
# a=cs_uber_dfs[cs_uber_dfs['car_number']=='MH01DR3092']
# a

In [ ]:
#car_status_report

sheet= client.open_by_key(car_status_report)
ws= sheet.worksheet('Cars')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dfs=dfm.loc[dfm['Type'].isin(['Revenue Share'])]
dfs['Current DM']=dfs['Current DM'].str.title()
reject_df=dfs.loc[dfs['Current DM'] != 'Pune']
reject_df_1=reject_df.loc[reject_df['Current DM'] != 'Ev']
car_status_df=reject_df_1.iloc[:,[1,0,4]]
car_status_df['ETM']=car_status_df['ETM'].str.strip()
car_status_df.rename({'Current DM':'Team Name','Car Number':'car_number'},axis=1,inplace=True)
car_status_df

In [ ]:
# #commitment mapping Dashboard

# sheet= client.open_by_key(commitment_mapping_key)
# ws= sheet.worksheet('Dashboard Pasting')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# commitment_mapping_dashboard_df=dfm.iloc[:,[1,3]]
# commitment_mapping_dashboard_df.rename({'Car No':'car_number','SDM Name':'Fleet_Lead'},axis=1,inplace=True)
# commitment_mapping_dashboard_df

all_car_details='C:\\Users\\sagar\\Dropbox\\DM Dashboard\\All Cars Details.xlsx'
all_car_details_df=pd.read_excel(all_car_details,sheet_name='Fleet_Lead')
commitment_mapping_dashboard_df=all_car_details_df.iloc[:,[1,2]]
commitment_mapping_dashboard_df.rename({'Car No':'car_number'},axis=1,inplace=True)
commitment_mapping_dashboard_df

In [ ]:
# #aatmanirbhar3

# sheet= client.open_by_key(aatmanirbhar3)
# ws= sheet.worksheet('Fleet_Mapping')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# aatmanirbhar_3_df=dfm.iloc[:,[0,1]]
# aatmanirbhar_3_df.rename({'License plate':'car_number'},axis=1,inplace=True)
# aatmanirbhar_3_df
all_car_details_df=pd.read_excel(all_car_details,sheet_name='Fleet_Lead')
aatmanirbhar_3_df=all_car_details_df.iloc[:,[1,3]]
aatmanirbhar_3_df.rename({'Car No':'car_number','SDM Name':'Fleet_Lead'},axis=1,inplace=True)
aatmanirbhar_3_df

In [ ]:
#aatmanirbhar3

all_car_details_df=pd.read_excel(all_car_details,sheet_name='Fleet_Lead')
aatmanirbhar_3_df=all_car_details_df.iloc[:,[1,2]]
aatmanirbhar_3_df.rename({'Car No':'car_number'},axis=1,inplace=True)
aatmanirbhar_3_df

#commitment mapping Dashboard

all_car_details='C:\\Users\\sagar\\Dropbox\\DM Dashboard\\All Cars Details.xlsx'
all_car_details_df=pd.read_excel(all_car_details,sheet_name='Fleet_Lead')
commitment_mapping_dashboard_df=all_car_details_df.iloc[:,[1,3]]
commitment_mapping_dashboard_df.rename({'Car No':'car_number','SDM Name':'Fleet_Lead'},axis=1,inplace=True)
commitment_mapping_dashboard_df

In [ ]:
#uber_df_today_yesterday

uber_df = pd.read_excel(uber_ws,sheet_name=-1)#uber sheet
today = datetime.date.today()
monday = today - datetime.timedelta(days=today.weekday())
today_timestmp_column = today.strftime("%Y-%m-%d %H:%M:%S")
ttc=today_timestmp_column+'.1'
yesterday = today- timedelta(days=1)
yesterday_timestmp_column=yesterday.strftime("%Y-%m-%d %H:%M:%S")
ytc=yesterday_timestmp_column+'.1'
uber_dfs=uber_df.loc[:,['Car No','Last Week Trips',(ytc),'Total Trips','Balance','Hours Online','Net Fare']]
uber_dfs.rename({'Car No':'car_number',(ytc):'Yesterdays Trips','Balance':'Balanced Trips','Hours Online':'Online Hours','Net Fare':'Revenue Total'},axis=1,inplace=True)
uber_dfs

#uber_dfs

cs_uber_dfs=car_status_df.merge(uber_dfs, on='car_number',how='left')
cs_uber_dfs_df=cs_uber_dfs.iloc[:,[3,4,5,6,7,8]]
cs_uber_dfs_df
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='N',end='S')
ws.set_dataframe(cs_uber_dfs_df,(1,14))

In [ ]:
#car_status_report

sheet= client.open_by_key(car_status_report)
ws= sheet.worksheet('Cars')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
dfs=dfm.loc[dfm['Type'].isin(['Revenue Share'])]
dfs['Current DM']=dfs['Current DM'].str.title()
reject_df=dfs.loc[dfs['Current DM'] != 'Pune']
reject_df_1=reject_df.loc[reject_df['Current DM'] != 'Ev']
car_status_df=reject_df_1.iloc[:,[1,0,4]]
car_status_df['ETM']=car_status_df['ETM'].str.strip()
car_status_df.rename({'Current DM':'Team Name','Car Number':'car_number'},axis=1,inplace=True)
car_status_df

#car_status_df 1 columns

car_status_df_1_column=car_status_df.iloc[:,[2]]
car_status_df_1_column
# sheet= clients.open_by_key(commitment_mapping_3)
# ws= sheet.worksheet_by_title('Master')
# ws.clear(start='E',end='E')
# ws.set_dataframe(car_status_df_1_column,(1,5))

In [ ]:
#commitment mapping Dashboard

sheet= client.open_by_key(commitment_mapping_key)
ws= sheet.worksheet('Dashboard Pasting')
data = ws.get_all_values()
headers = data.pop(0)
dfm = pd.DataFrame(data,columns=headers)
commitment_mapping_dashboard_df=dfm.iloc[:,[1,3]]
commitment_mapping_dashboard_df.rename({'Car No':'car_number','SDM Name':'Fleet_Lead'},axis=1,inplace=True)
commitment_mapping_dashboard_df



In [ ]:
# #car_status_report

# sheet= client.open_by_key(car_status_report)
# ws= sheet.worksheet('Cars')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# dfs=dfm.loc[dfm['Type'].isin(['Revenue Share'])]
# reject_df=dfs.loc[dfs['Current DM'] != 'Pune']
# car_status_df=reject_df.iloc[:,[1,0,4]]
# car_status_df['ETM']=car_status_df['ETM'].str.strip()
# car_status_df.rename({'Current DM':'Team Name'},axis=1,inplace=True)
# car_status_df

In [ ]:
reject_df['Current DM'].unique()

In [ ]:
############################################################ pushing specific team data to 5 dm sheets

#pushing car status 2 columns to all 5 dm sheets



car_status_df_2_column=car_status_df.iloc[:,[0,1]]
car_status_df_2_column_terrific=car_status_df_2_column.loc[car_status_df_2_column['Team Name'].isin(['Terrific Tigers'])]
car_status_df_2_column_terrific
sheet= clients.open_by_key(terrific)
ws= sheet.worksheet_by_title('Terrific_Tigers')
ws.clear(start='B',end='C')
ws.set_dataframe(car_status_df_2_column_terrific,(1,2))

car_status_df_2_column=car_status_df.iloc[:,[0,1]]
car_status_df_2_column_roaring=car_status_df_2_column.loc[car_status_df_2_column['Team Name'].isin(['Roaring Lions'])]
car_status_df_2_column_roaring
sheet= clients.open_by_key(roaring)
ws= sheet.worksheet_by_title('Roaring_Lions')
ws.clear(start='B',end='C')
ws.set_dataframe(car_status_df_2_column_roaring,(1,2))


car_status_df_2_column=car_status_df.iloc[:,[0,1]]
car_status_df_2_column_silent=car_status_df_2_column.loc[car_status_df_2_column['Team Name'].isin(['Silent Killers'])]
car_status_df_2_column_silent
sheet= clients.open_by_key(silent)
ws= sheet.worksheet_by_title('Silent_Killers')
ws.clear(start='B',end='C')
ws.set_dataframe(car_status_df_2_column_silent,(1,2))

car_status_df_2_column=car_status_df.iloc[:,[0,1]]
car_status_df_2_column_deep=car_status_df_2_column.loc[car_status_df_2_column['Team Name'].isin(['Deep Hunters'])]
car_status_df_2_column_deep
sheet= clients.open_by_key(deep)
ws= sheet.worksheet_by_title('Deep_Hunters')
ws.clear(start='B',end='C')
ws.set_dataframe(car_status_df_2_column_deep,(1,2))

car_status_df_2_column=car_status_df.iloc[:,[0,1]]
car_status_df_2_column_black=car_status_df_2_column.loc[car_status_df_2_column['Team Name'].isin(['Black Panthers'])]
car_status_df_2_column_black
sheet= clients.open_by_key(black)
ws= sheet.worksheet_by_title('Black_Panthers')
ws.clear(start='B',end='C')
ws.set_dataframe(car_status_df_2_column_black,(1,2))

#pushing car status 1 column to all dm sheets

car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_terrific=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Terrific Tigers'])]
car_status_df_1_column_terrific_df=car_status_df_1_column_terrific.iloc[:,[2]]
car_status_df_1_column_terrific_df
sheet= clients.open_by_key(terrific)
ws= sheet.worksheet_by_title('Terrific_Tigers')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_terrific_df,(1,5))

car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_roaring=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Roaring Lions'])]
car_status_df_1_column_roaring_df=car_status_df_1_column_roaring.iloc[:,[2]]
car_status_df_1_column_roaring_df
sheet= clients.open_by_key(roaring)
ws= sheet.worksheet_by_title('Roaring_Lions')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_roaring_df,(1,5))

car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_silent=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Silent Killers'])]
car_status_df_1_column_silent_df=car_status_df_1_column_silent.iloc[:,[2]]
car_status_df_1_column_silent_df
sheet= clients.open_by_key(silent)
ws= sheet.worksheet_by_title('Silent_Killers')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_silent_df,(1,5))

car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_deep=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Deep Hunters'])]
car_status_df_1_column_deep_df=car_status_df_1_column_deep.iloc[:,[2]]
car_status_df_1_column_deep_df
sheet= clients.open_by_key(deep)
ws= sheet.worksheet_by_title('Deep_Hunters')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_deep_df,(1,5))

car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_black=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Black Panthers'])]
car_status_df_1_column_black_df=car_status_df_1_column_black.iloc[:,[2]]
car_status_df_1_column_black_df
sheet= clients.open_by_key(black)
ws= sheet.worksheet_by_title('Black_Panthers')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_black_df,(1,5))

In [ ]:
# Terrific Tigers
# Roaring Lions
# Silent Killers
# Deep Hunters
# Black Panthers

# terrific='1f-DJ5O3zKKkAtXfXQghWzbsoXyy-ipdJ5q7Yma4tF9M'
# roaring='1MiZZY9MPEhhg-B9LyfmThQGV3Dy3XnKpb5NqgaYFwIQ'
# silent='1ZwXyZPyt7qhjTO5VkfwoGTJ48fmg_38MTdy4EHdVhwA'
# deep='1TPRKPsQfy4qY19byAuYi0Ie3Xld7ddrrBS8xqhggtcM'
# black

Terrific_Tigers
Roaring_Lions
Silent_Killers
Deep_Hunters
Black_Panthers

In [ ]:
#commitment dashboard

cs_cmdf=car_status_df_2_column_terrific.merge(commitment_mapping_dashboard_df, on='car_number',how='left')
cs_cmdf_df_terrfic=cs_cmdf.iloc[:,[2]]
cs_cmdf_df_terrfic
# sheet= clients.open_by_key(terrific)
# ws= sheet.worksheet_by_title('Terrific_Tigers')
# ws.clear(start='A',end='A')
# ws.set_dataframe(cs_cmdf_df_terrfic,(1,1))

# cs_cmdf=car_status_df_2_column_roaring.merge(commitment_mapping_dashboard_df, on='car_number',how='left')
# cs_cmdf_df_roaring=cs_cmdf.iloc[:,[2]]
# cs_cmdf_df_roaring
# sheet= clients.open_by_key(roaring)
# ws= sheet.worksheet_by_title('Roaring_Lions')
# ws.clear(start='A',end='A')
# ws.set_dataframe(cs_cmdf_df_roaring,(1,1))

# cs_cmdf=car_status_df_2_column_black.merge(commitment_mapping_dashboard_df, on='car_number',how='left')
# cs_cmdf_df_terrfic=cs_cmdf.iloc[:,[2]]
# cs_cmdf_df_silent
# sheet= clients.open_by_key(silent)
# ws= sheet.worksheet_by_title('Silent_Killers')
# ws.clear(start='A',end='A')
# ws.set_dataframe(cs_cmdf_df_terrfic,(1,1))

# cs_cmdf=car_status_df_2_column_black.merge(commitment_mapping_dashboard_df, on='car_number',how='left')
# cs_cmdf_df_terrfic=cs_cmdf.iloc[:,[2]]
# cs_cmdf_df_terrfic
# sheet= clients.open_by_key(deep)
# ws= sheet.worksheet_by_title('Deep_Hunters')
# ws.clear(start='A',end='A')
# ws.set_dataframe(cs_cmdf_df_terrfic,(1,1))

# cs_cmdf=car_status_df_2_column_black.merge(commitment_mapping_dashboard_df, on='car_number',how='left')
# cs_cmdf_df_black=cs_cmdf.iloc[:,[2]]
# cs_cmdf_df_black
# sheet= clients.open_by_key(black)
# ws= sheet.worksheet_by_title('Black_Panthers')
# ws.clear(start='A',end='A')
# ws.set_dataframe(cs_cmdf_df_terrfic,(1,1))


In [ ]:
car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_terrific=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Terrific Tigers'])]
car_status_df_1_column_terrific_df=car_status_df_1_column_terrific.iloc[:,[2]]
car_status_df_1_column_terrific_df
sheet= clients.open_by_key(terrific)
ws= sheet.worksheet_by_title('Terrific_Tigers')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_terrific_df,(1,5))

car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_roaring=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Roaring Lions'])]
car_status_df_1_column_roaring_df=car_status_df_1_column_roaring.iloc[:,[2]]
car_status_df_1_column_roaring_df
sheet= clients.open_by_key(roaring)
ws= sheet.worksheet_by_title('Roaring_Lions')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_roaring_df,(1,5))

car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_silent=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Silent Killers'])]
car_status_df_1_column_silent_df=car_status_df_1_column_silent.iloc[:,[2]]
car_status_df_1_column_silent_df
sheet= clients.open_by_key(silent)
ws= sheet.worksheet_by_title('Silent_Killers')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_silent_df,(1,5))

car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_deep=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Deep Hunters'])]
car_status_df_1_column_deep_df=car_status_df_1_column_deep.iloc[:,[2]]
car_status_df_1_column_deep_df
sheet= clients.open_by_key(deep)
ws= sheet.worksheet_by_title('Deep_Hunters')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_deep_df,(1,5))

car_status_df_1_column=car_status_df.iloc[:,[0,1,2]]
car_status_df_1_column_black=car_status_df_1_column.loc[car_status_df_1_column['Team Name'].isin(['Black Panthers'])]
car_status_df_1_column_black_df=car_status_df_1_column_black.iloc[:,[2]]
car_status_df_1_column_black_df
sheet= clients.open_by_key(black)
ws= sheet.worksheet_by_title('Black_Panthers')
ws.clear(start='E',end='E')
ws.set_dataframe(car_status_df_1_column_black_df,(1,5))

In [ ]:
a='qwret wrte'
a.capitalize()

In [16]:
pd.set_option('display.max_columns', None)

In [ ]:
# #car_servicing_schedule_calling_servicing_tab

# sheet= client.open_by_key(car_servicing_schedule_calling_servicing_tab)
# ws= sheet.worksheet('Calling Servicing Pending')
# data = ws.get_all_values()
# headers = data.pop(0)
# dfm = pd.DataFrame(data,columns=headers)
# servicing_df=dfm.iloc[:,[1,0]]
# servicing_df.rename({'Car Number':'car_number','':'Servicing'},axis=1,inplace=True)
# servicing_df

In [ ]:
lwdk_df=car_status_df.merge(Last_Week_Dead_KM, on='car_number',how='left')
lwdk_dfs=lwdk_df.iloc[:,[3]]
sheet= clients.open_by_key(commitment_mapping_3)
ws= sheet.worksheet_by_title('Master')
ws.clear(start='Y',end='Y')
ws.set_dataframe(lwdk_dfs,(1,25))

In [ ]:
#importing libraries

import pandas as pd
import pandasql as ps
import datetime
from datetime import date, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets
import warnings
warnings.filterwarnings("ignore")

#creating connections

scope= ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials= ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope) 
client= gspread.authorize(credentials)

clients = pygsheets.authorize(service_file='client_secret.json')


#sheet keys

car_status_report='1CfqvArNmTofvNOFAhQ965B7cMA7lX40e3RZZKY6IUjI'
commitment_mapping_key='14zYk5KJ5J8uOxTlHvFF5uf-h_dRUa98x7FOl0AdbvIk'
aatmanirbhar3='1cC9dKh4tvSHQuL5ncSJgxD79IyeYlVYwSx-mU1Hz3fI'
fleet_driver='1qceRS8LU17n5YWvgewcpJZif02KxV8Mtvs6anomHD80'
allotment_status_report='1cpR6AVVpk9TF4_I38IFYPPOqk-_bSROHgVVYdaXLXOI'
car_master='1_r5OMN1P8Tof5IRaE5jYd-jHaP8j15avQVMycDJdhec'
uber_ws='C:\\Users\\sagar\\Dropbox\\DM Dashboard\\Master View.xlsx' 
dps_ws='C:\\Users\\sagar\\Dropbox\\DM Dashboard\\Driver Performance Sheet.xlsx'
car_servicing_schedule_calling_servicing_tab='11WVBiisNIF8Xb7sx7GZEok7mG4Lz3uo96-e00xVtW0M'
commitment_mapping_3='1sy3Gxrnh8bX6ibpGR6X8rba8H2IfeSNdBGT7CIFi3SY'

#5 sheets keys

terrific='1f-DJ5O3zKKkAtXfXQghWzbsoXyy-ipdJ5q7Yma4tF9M'
roaring='1MiZZY9MPEhhg-B9LyfmThQGV3Dy3XnKpb5NqgaYFwIQ'
silent='1ZwXyZPyt7qhjTO5VkfwoGTJ48fmg_38MTdy4EHdVhwA'
deep='1TPRKPsQfy4qY19byAuYi0Ie3Xld7ddrrBS8xqhggtcM'
black='1KD0ABDWp3YvqIkmoQAZKRS-7-S8i-HtlO4KKpBMB1tU'